## Modelo en SKLearn

In [ ]:
lrmodel = LogisticRegression()

## Definición de Parámetros

In [ ]:
solvers = ['newton-cg','liblinear','lbfgs']
penalty = ['l2','l1', 'elasticnet']
c_values = [100, 10, 1.0, 0.1, 0.01]

## Espacio de Búsqueda de Hiperparámetros

In [ ]:
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=5,
                             n_repeats=5,
                             random_state=1)

grid_search = GridSearchCV(estimator=lrmodel,
                           param_grid=grid,
                           n_jobs=-1, cv=3,
                           scoring='accuracy',
                           error_score=0)

grid_result = grid_search.fit(principal_components, y)

print("Best: %f using %s" % (grid_result.best_score_,
                             grid_result.best_params_))

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

## Modelo Final

In [ ]:
logreg = LogisticRegression(solver = 'liblinear',
                            penalty='l2',
                            C=100)
logreg.fit(X_train_pca, y_train)

## Predicción

In [ ]:
y_pred_lr = logreg.predict(X_test_pca)
print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(logreg.score(X_train_pca,
                                                                                            y_train)))

print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test_pca,
                                                                                           y_test)))

print('Precision of logistic regression classifier on test set: {:.2f}'.format(precision_score(y_test,
                                                                                               y_pred_lr)))

### Métricas

In [ ]:
accuracy_lr = accuracy_score(y_test, y_pred_lr)
precision_lr = precision_score(y_test, y_pred_lr)
recall_lr = recall_score(y_test, y_pred_lr)
f1_score_lr = f1_score(y_test, y_pred_lr)
conf_matrix = confusion_matrix(y_test, y_pred_lr)
print(accuracy_lr, precision_lr, recall_lr, f1_score_lr, conf_matrix)

In [ ]:
y_scores_lr = logreg.predict_proba(X_test_pca)[:, 1]
fpr_lr, tpr_lr, thresholds_lr = roc_curve(y_test, y_scores_lr)
auc_roc_lr = roc_auc_score(y_test, y_scores_lr)

plt.figure(figsize=(8, 8))
plt.plot(fpr_lr, tpr_lr, color='blue', lw=2, label=f'AUC = {auc_roc_lr:.2f}')
plt.plot([0, 1], [0, 1], color='green', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos (1 - Especificidad)')
plt.ylabel('Tasa de Verdaderos Positivos (Sensibilidad)')
plt.title('Curva ROC')
plt.legend(loc="lower right")
plt.show()